# **Import Object**

In [1]:
from datetime import datetime as dt
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import get_x_y

# **Load Energy**

In [2]:
energy_7_9_60m = get_x_y.get_target_energy(house_id=4,
                                           target_months=[7, 8, 9],
                                           intervals=3600)
energy_11_1_60m = get_x_y.get_target_energy(4, [11, 12, 1], 3600)

energy_7_9_30m = get_x_y.get_target_energy(4, [7, 8, 9], 1800)
energy_11_1_30m = get_x_y.get_target_energy(4, [11, 12, 1], 1800)

# **Load Occupancy**

In [3]:
occupancy_sum = pd.read_csv('./ecodataset/Occupancy/04_summer.csv')
occupancy_sum = occupancy_sum.set_index('Unnamed: 0')

occupancy_win = pd.read_csv('./ecodataset/Occupancy/04_winter.csv')
occupancy_win = occupancy_win.set_index('Unnamed: 0')

In [4]:
occupancy_sum = get_x_y.to_intervals_occupancy(occupancies=occupancy_sum,
                                               intervals=1800)
occupancy_win = get_x_y.to_intervals_occupancy(occupancy_win, 1800)

# **Create Ratio**

In [5]:
ratio_sum = get_x_y.build_ratio(date_columns=occupancy_sum.columns,
                                energies=energy_7_9_60m)
ratio_win = get_x_y.build_ratio(occupancy_win.columns, energy_11_1_60m)

# **Create Features**

In [6]:
energy_sum = get_x_y.get_corresponding_energy(
    occupancy_columns=occupancy_sum.columns,
    energies=energy_7_9_30m)

In [7]:
energy_win = get_x_y.get_corresponding_energy(occupancy_win.columns,
                                              energy_11_1_30m)

In [8]:
features_sum = pd.DataFrame()
features_sum["Energy"] = energy_sum
features_sum["Season"] = 1
features_sum["Time"] = [t for t in range(0, 48, 1)] * len(occupancy_sum.columns)
features_sum = features_sum[(features_sum.Time >= 12) & (features_sum.Time < 44)]

features_win = pd.DataFrame()
features_win["Energy"] = energy_win
features_win["Season"] = 0
features_win["Time"] = [t for t in range(0, 48, 1)] * len(occupancy_win.columns)
features_win = features_win[(features_win.Time >= 12) & (features_win.Time < 44)]

In [9]:
occupancy_sum = occupancy_sum[12:44].values.T.reshape(-1)
occupancy_sum = pd.DataFrame(occupancy_sum, columns=["Occupancy"])

occupancy_win = occupancy_win[12:44].values.T.reshape(-1)
occupancy_win = pd.DataFrame(occupancy_win, columns=["Occupancy"])

In [10]:
X = pd.concat([features_sum, features_win], axis=0)
Y = pd.concat([occupancy_sum, occupancy_win], axis=0)

In [11]:
X.to_csv("4_X_train.csv", index=False)
Y.to_csv("4_Y_train.csv", index=False)